## Odwiedzający (ang. Visitor)

**Typ**: behawioralny \
**Zakres**: obiektowy

<div style="border: solid 1px;padding: 20px;text-align: center">
    Wzorzec <b>odwiedzający</b> pozwala definiować nowe operacje bez zmiany klas elementów, na których te operacje są wykonywane.
</div>

### Problem - dodawanie nowych operacji do hierarchii klas

Mamy kształty geometryczne: Circle, Rectangle, Triangle. Chcemy:
- Rysować je (`draw()`)
- Liczyć pole (`calculate_area()`)
- Eksportować do JSON (`export_json()`)

**Problem:**
- Każda nowa operacja = modyfikacja każdej klasy
- Operacje rozproszone po wszystkich klasach
- Trudno dodać operację bez modyfikacji istniejących klas

### Naiwne podejście - operacje w klasach

In [ ]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    def draw(self):
        print(f"🔵 Rysowanie koła o promieniu {self.radius}")
    
    def calculate_area(self):
        return math.pi * self.radius ** 2
    
    # ❌ Nowa operacja = trzeba zmodyfikować klasę!
    def export_json(self):
        return f'{{"type": "circle", "radius": {self.radius}}}'


class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def draw(self):
        print(f"🟦 Rysowanie prostokąta {self.width}x{self.height}")
    
    def calculate_area(self):
        return self.width * self.height
    
    # ❌ Nowa operacja = modyfikacja każdej klasy!
    def export_json(self):
        return f'{{"type": "rectangle", "width": {self.width}, "height": {self.height}}}'

In [ ]:
circle = Circle(5)
rectangle = Rectangle(10, 20)

circle.draw()
print(f"Pole: {circle.calculate_area():.2f}")
print(f"JSON: {circle.export_json()}")

print()
rectangle.draw()
print(f"Pole: {rectangle.calculate_area()}")
print(f"JSON: {rectangle.export_json()}")

**Problemy:**
- ❌ **Naruszenie Open/Closed** - każda nowa operacja = modyfikacja klas
- ❌ **Operacje rozproszone** - `draw()` w Circle, Rectangle, Triangle...
- ❌ **Trudność dodawania operacji** - trzeba zmieniać wszystkie klasy
- ❌ **Brak separacji** - logika operacji zmieszana z klasami

### Rozwiązanie - wzorzec Odwiedzający

**Idea:** Wydziel operacje do osobnych klas (Visitors). Każdy element **akceptuje** visitora.

**Kluczowy mechanizm: Double Dispatch**
```python
element.accept(visitor)  # Element wie jaki jest (Circle, Rectangle)
    ↓
    visitor.visit_circle(self)  # Visitor wie co zrobić z Circle
```

### Krok 1: Interfejs Visitor

In [ ]:
from abc import ABC, abstractmethod

class ShapeVisitor(ABC):
    """Interfejs visitora - dla każdego typu osobna metoda"""
    
    @abstractmethod
    def visit_circle(self, circle):
        pass
    
    @abstractmethod
    def visit_rectangle(self, rectangle):
        pass
    
    @abstractmethod
    def visit_triangle(self, triangle):
        pass

### Krok 2: Elementy - klasy z metodą accept()

In [ ]:
import math

class Shape(ABC):
    """Bazowy element - MUSI mieć accept()"""
    
    @abstractmethod
    def accept(self, visitor):
        pass


class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def accept(self, visitor: ShapeVisitor):
        """Akceptuj visitora - double dispatch"""
        return visitor.visit_circle(self)  # Przekaż SIEBIE do visitora


class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def accept(self, visitor: ShapeVisitor):
        return visitor.visit_rectangle(self)


class Triangle(Shape):
    def __init__(self, base, height):
        self.base = base
        self.height = height
    
    def accept(self, visitor: ShapeVisitor):
        return visitor.visit_triangle(self)

**Kluczowa obserwacja:**
- Klasy kształtów **NIE zawierają** operacji (`draw()`, `calculate_area()`)
- Zawierają tylko **dane** i metodę `accept()`
- `accept()` przekazuje `self` do odpowiedniej metody visitora

### Krok 3: ConcreteVisitor #1 - Rysowanie

In [ ]:
class DrawVisitor(ShapeVisitor):
    """Visitor dla operacji rysowania"""
    
    def visit_circle(self, circle):
        print(f"🔵 Rysowanie koła o promieniu {circle.radius}")
    
    def visit_rectangle(self, rectangle):
        print(f"🟦 Rysowanie prostokąta {rectangle.width}x{rectangle.height}")
    
    def visit_triangle(self, triangle):
        print(f"🔺 Rysowanie trójkąta o podstawie {triangle.base}")

### Krok 4: ConcreteVisitor #2 - Liczenie pola

In [ ]:
class AreaVisitor(ShapeVisitor):
    """Visitor dla operacji liczenia pola"""
    
    def visit_circle(self, circle):
        return math.pi * circle.radius ** 2
    
    def visit_rectangle(self, rectangle):
        return rectangle.width * rectangle.height
    
    def visit_triangle(self, triangle):
        return 0.5 * triangle.base * triangle.height

### Krok 5: ConcreteVisitor #3 - Eksport JSON

In [ ]:
class JsonExportVisitor(ShapeVisitor):
    """Visitor dla eksportu do JSON"""
    
    def visit_circle(self, circle):
        return f'{{"type": "circle", "radius": {circle.radius}}}'
    
    def visit_rectangle(self, rectangle):
        return f'{{"type": "rectangle", "width": {rectangle.width}, "height": {rectangle.height}}}'
    
    def visit_triangle(self, triangle):
        return f'{{"type": "triangle", "base": {triangle.base}, "height": {triangle.height}}}'

### Krok 6: Użycie - różne operacje bez modyfikacji klas

In [ ]:
# Tworzenie kształtów
shapes = [
    Circle(5),
    Rectangle(10, 20),
    Triangle(8, 6)
]

# Tworzenie visitorów
draw_visitor = DrawVisitor()
area_visitor = AreaVisitor()
json_visitor = JsonExportVisitor()

# Wykonywanie operacji
for shape in shapes:
    shape.accept(draw_visitor)  # Rysuj
    area = shape.accept(area_visitor)  # Licz pole
    json_str = shape.accept(json_visitor)  # Eksportuj
    print(f"  Pole: {area:.2f}")
    print(f"  JSON: {json_str}")
    print()

**Zalety:**
- ✅ **Open/Closed** - nowa operacja = nowy visitor (nie modyfikacja klas!)
- ✅ **Single Responsibility** - każdy visitor = jedna operacja
- ✅ **Łatwe dodawanie operacji** - np. `XmlExportVisitor`, `PerimeterVisitor`
- ✅ **Grupowanie operacji** - cała logika rysowania w `DrawVisitor`

## Jak to działa? - Double Dispatch

```python
circle = Circle(5)
draw_visitor = DrawVisitor()

circle.accept(draw_visitor)
    ↓
    # W Circle.accept()
    return visitor.visit_circle(self)  # self = circle
        ↓
        # W DrawVisitor.visit_circle()
        print(f"Rysowanie koła o promieniu {circle.radius}")
```

**Double Dispatch = dwa "dispatche":**
1. **Pierwszy dispatch**: Który `accept()` wywołać? → Zależy od typu obiektu (Circle, Rectangle)
2. **Drugi dispatch**: Który `visit_xxx()` wywołać? → Zależy od typu visitora (DrawVisitor, AreaVisitor)

**Dzięki temu:**
- Element wie **jaki jest** (Circle wywołuje `visit_circle()`)
- Visitor wie **co zrobić** z tym elementem

## Struktura wzorca

**Elementy wzorca Odwiedzający:**

1. **Visitor** - `ShapeVisitor`
   - Interfejs dla visitorów
   - Metoda `visit_xxx()` dla **każdego** typu elementu

2. **ConcreteVisitor** - `DrawVisitor`, `AreaVisitor`
   - Implementuje operację dla każdego typu
   - Zna strukturę wszystkich elementów

3. **Element** - `Shape`
   - Interfejs z metodą `accept(visitor)`

4. **ConcreteElement** - `Circle`, `Rectangle`
   - Implementuje `accept()` → wywołuje odpowiedni `visit_xxx(self)`

**Kluczowa właściwość:**
> Nowa operacja = nowy Visitor (bez modyfikacji elementów)

## Przykład 2 - Eksport dokumentu

In [ ]:
from abc import ABC, abstractmethod

# ════════════════════════════════════════════════════════════
# Visitor
# ════════════════════════════════════════════════════════════
class DocumentVisitor(ABC):
    @abstractmethod
    def visit_paragraph(self, paragraph):
        pass
    
    @abstractmethod
    def visit_heading(self, heading):
        pass
    
    @abstractmethod
    def visit_image(self, image):
        pass


# ════════════════════════════════════════════════════════════
# Elements
# ════════════════════════════════════════════════════════════
class DocumentElement(ABC):
    @abstractmethod
    def accept(self, visitor):
        pass


class Paragraph(DocumentElement):
    def __init__(self, text):
        self.text = text
    
    def accept(self, visitor):
        return visitor.visit_paragraph(self)


class Heading(DocumentElement):
    def __init__(self, level, text):
        self.level = level
        self.text = text
    
    def accept(self, visitor):
        return visitor.visit_heading(self)


class Image(DocumentElement):
    def __init__(self, url, alt):
        self.url = url
        self.alt = alt
    
    def accept(self, visitor):
        return visitor.visit_image(self)


# ════════════════════════════════════════════════════════════
# ConcreteVisitors - różne formaty eksportu
# ════════════════════════════════════════════════════════════
class HtmlExportVisitor(DocumentVisitor):
    """Eksport do HTML"""
    
    def visit_paragraph(self, paragraph):
        return f"<p>{paragraph.text}</p>"
    
    def visit_heading(self, heading):
        return f"<h{heading.level}>{heading.text}</h{heading.level}>"
    
    def visit_image(self, image):
        return f'<img src="{image.url}" alt="{image.alt}" />'


class MarkdownExportVisitor(DocumentVisitor):
    """Eksport do Markdown"""
    
    def visit_paragraph(self, paragraph):
        return paragraph.text
    
    def visit_heading(self, heading):
        return "#" * heading.level + f" {heading.text}"
    
    def visit_image(self, image):
        return f"![{image.alt}]({image.url})"


class PlainTextVisitor(DocumentVisitor):
    """Eksport do czystego tekstu"""
    
    def visit_paragraph(self, paragraph):
        return paragraph.text
    
    def visit_heading(self, heading):
        return heading.text.upper()  # Nagłówki dużymi literami
    
    def visit_image(self, image):
        return f"[Obraz: {image.alt}]"  # Tekst zastępczy

In [ ]:
# Dokument
document = [
    Heading(1, "Tytuł dokumentu"),
    Paragraph("To jest pierwszy akapit."),
    Heading(2, "Sekcja"),
    Paragraph("Drugi akapit z treścią."),
    Image("photo.jpg", "Przykładowe zdjęcie")
]

# Eksport do różnych formatów
html_visitor = HtmlExportVisitor()
markdown_visitor = MarkdownExportVisitor()
text_visitor = PlainTextVisitor()

print("=== HTML ===")
for element in document:
    print(element.accept(html_visitor))

print("\n=== Markdown ===")
for element in document:
    print(element.accept(markdown_visitor))

print("\n=== Plain Text ===")
for element in document:
    print(element.accept(text_visitor))

**Kluczowe:**
- Dodanie nowego formatu (np. PDF) = nowy visitor
- **NIE** trzeba modyfikować `Paragraph`, `Heading`, `Image`
- Cała logika eksportu w jednym miejscu (visitor)

## Visitor vs Strategy

| Aspekt | Visitor | Strategy |
|--------|---------|----------|
| **Cel** | Operacje na **wielu typach** obiektów | **Jedna** operacja, różne algorytmy |
| **Liczba typów** | Wiele (Circle, Rectangle, Triangle) | Jeden typ |
| **Dispatch** | Double dispatch (`accept()` + `visit_xxx()`) | Single dispatch (strategia wybrana z góry) |
| **Struktura** | Visitor zna **wszystkie** typy elementów | Strategy nie zna kontekstu |
| **Przykład** | Export do wielu formatów dla różnych elementów | Sortowanie jednej listy (różne algorytmy) |

**Kluczowa różnica:**
```python
# Visitor - WIELE typów
visitor.visit_circle(circle)      # Osobna metoda dla Circle
visitor.visit_rectangle(rect)     # Osobna metoda dla Rectangle
visitor.visit_triangle(triangle)  # Osobna metoda dla Triangle

# Strategy - JEDEN typ
strategy.sort(list)  # Ta sama metoda dla wszystkich list
```

## Kiedy używać wzorca Odwiedzający?

Wzorzec Odwiedzający stosuj gdy:

1. **Często dodajesz nowe operacje, rzadko nowe klasy**
   - Nowa operacja = nowy visitor (łatwo)
   - Nowa klasa = modyfikacja wszystkich visitorów (trudno)

2. **Operacje są niepowiązane ze sobą**
   - Rysowanie, eksport, walidacja - różne domeny

3. **Chcesz zgrupować operacje**
   - Cała logika eksportu w `ExportVisitor`

4. **Masz stabilną hierarchię klas**
   - Typy elementów się nie zmieniają często

**Przykłady praktyczne:**
- Kompilatory - AST (Abstract Syntax Tree) operations
- Eksport dokumentów do różnych formatów
- Walidacja złożonych struktur danych
- Rendering grafiki (SVG, Canvas, WebGL)
- Kalkulacje na strukturach (wyrażenia matematyczne)

## Wady wzorca Odwiedzający

⚠️ **Trudność dodawania nowych klas:**
- Nowa klasa (np. `Pentagon`) wymaga dodania `visit_pentagon()` do **wszystkich** visitorów

⚠️ **Naruszenie enkapsulacji:**
- Visitor musi mieć dostęp do wewnętrznych pól elementów
- Czasami wymaga getterów dla prywatnych pól

⚠️ **Złożoność:**
- Double dispatch może być trudny do zrozumienia
- Wymaga zmiany myślenia

## Przykład 3 - Struktura katalogów

In [ ]:
from abc import ABC, abstractmethod

# Visitor
class FileSystemVisitor(ABC):
    @abstractmethod
    def visit_file(self, file):
        pass
    
    @abstractmethod
    def visit_directory(self, directory):
        pass


# Elements
class FileSystemElement(ABC):
    @abstractmethod
    def accept(self, visitor):
        pass


class File(FileSystemElement):
    def __init__(self, name, size):
        self.name = name
        self.size = size  # KB
    
    def accept(self, visitor):
        return visitor.visit_file(self)


class Directory(FileSystemElement):
    def __init__(self, name):
        self.name = name
        self.children = []
    
    def add(self, element):
        self.children.append(element)
    
    def accept(self, visitor):
        return visitor.visit_directory(self)


# ConcreteVisitor #1 - Rozmiar
class SizeCalculatorVisitor(FileSystemVisitor):
    """Oblicza całkowity rozmiar"""
    
    def visit_file(self, file):
        return file.size
    
    def visit_directory(self, directory):
        total = 0
        for child in directory.children:
            total += child.accept(self)  # Rekursywnie
        return total


# ConcreteVisitor #2 - Wyświetlanie
class DisplayVisitor(FileSystemVisitor):
    """Wyświetla strukturę drzewa"""
    
    def __init__(self):
        self.indent = 0
    
    def visit_file(self, file):
        print("  " * self.indent + f"📄 {file.name} ({file.size} KB)")
    
    def visit_directory(self, directory):
        print("  " * self.indent + f"📁 {directory.name}/")
        self.indent += 1
        for child in directory.children:
            child.accept(self)
        self.indent -= 1


# ConcreteVisitor #3 - Wyszukiwanie
class SearchVisitor(FileSystemVisitor):
    """Szuka plików z rozszerzeniem"""
    
    def __init__(self, extension):
        self.extension = extension
        self.results = []
    
    def visit_file(self, file):
        if file.name.endswith(self.extension):
            self.results.append(file.name)
    
    def visit_directory(self, directory):
        for child in directory.children:
            child.accept(self)

In [ ]:
# Budowanie struktury
root = Directory("project")
src = Directory("src")
src.add(File("main.py", 10))
src.add(File("utils.py", 5))

tests = Directory("tests")
tests.add(File("test_main.py", 3))
tests.add(File("test_utils.py", 2))

root.add(src)
root.add(tests)
root.add(File("README.md", 1))

# Visitor #1 - Wyświetlenie struktury
display_visitor = DisplayVisitor()
root.accept(display_visitor)

# Visitor #2 - Rozmiar
print()
size_visitor = SizeCalculatorVisitor()
total_size = root.accept(size_visitor)
print(f"Całkowity rozmiar: {total_size} KB")

# Visitor #3 - Wyszukiwanie
print()
search_visitor = SearchVisitor(".py")
root.accept(search_visitor)
print(f"Pliki .py: {search_visitor.results}")

**Obserwacje:**
- 3 operacje (wyświetlanie, rozmiar, szukanie) = 3 visitery
- Struktura katalogów **nie zmieniła się** przy dodawaniu operacji
- Visitor może być **rekursywny** (katalogi w katalogach)

## Podsumowanie

Wzorzec Odwiedzający:
- ✅ **Open/Closed** - nowe operacje bez modyfikacji klas
- ✅ **Single Responsibility** - każdy visitor = jedna operacja
- ✅ **Grupowanie logiki** - operacje zebrane w visitorach
- ✅ **Łatwość dodawania operacji** - nowy visitor
- ⚠️ **Trudność dodawania klas** - wymaga modyfikacji wszystkich visitorów
- ⚠️ **Naruszenie enkapsulacji** - visitor potrzebuje dostępu do pól
- ⚠️ **Złożoność** - double dispatch

**Kluczowa idea:**
> **Separacja operacji od struktury** - operacje w visitorach, dane w elementach

**Struktura:**
```python
# Visitor
class Visitor(ABC):
    @abstractmethod
    def visit_concrete_element_a(self, element):
        pass
    
    @abstractmethod
    def visit_concrete_element_b(self, element):
        pass

# Element
class Element(ABC):
    @abstractmethod
    def accept(self, visitor):
        pass

# ConcreteElement
class ConcreteElementA(Element):
    def accept(self, visitor):
        return visitor.visit_concrete_element_a(self)  # Double dispatch

# ConcreteVisitor
class ConcreteVisitor(Visitor):
    def visit_concrete_element_a(self, element):
        # Operacja na ElementA
        pass
    
    def visit_concrete_element_b(self, element):
        # Operacja na ElementB
        pass
```

**Istota wzorca:**
- **Double dispatch:** `element.accept(visitor)` → `visitor.visit_xxx(element)`
- **Separacja:** Operacje w visitorach, dane w elementach
- **Łatwość rozszerzania:** Nowa operacja = nowy visitor
- **Cena:** Trudno dodawać nowe typy elementów

**Kiedy stosować:**
- ✅ Często dodajesz operacje, rzadko typy
- ✅ Operacje niepowiązane ze sobą
- ✅ Stabilna hierarchia klas
- ❌ Często dodajesz nowe typy

**Kluczowe mechanizmy:**
1. `accept(visitor)` w każdym elemencie
2. `visit_xxx()` dla każdego typu w visitorze
3. Element wie **jaki jest**, Visitor wie **co zrobić**